# Generating gains for wav calibration on augmented soundscapes

In ARAUS dataset responses.csv constitute the dataset of +25k augmented soundscapes labeled with psychoacoustic and acoustic parametres. Among these, we can find Leq_r, which constitutes the Leq of channel R for each audio. These value is calculated according to the formula below. We calculate the Leq on the digital signal channel R, without any wav calibration. The obtained value is sustracted from the corresponding Leq_r. The difference is converted to lineal, and it constitutes the wav calibration we need to apply to the audio.

In [1]:
# Add MOSQITO to the Python path
import sys
import os
sys.path.append('..')

# Import numpy
import numpy as np
import pandas as pd
# Import plot function
import matplotlib.pyplot as plt
# Import mosqito functions
#from mosqito.utils import load
from scipy.io import wavfile
from scipy.signal import resample
from maad.spl import pressure2leq, pressure2dBSPL
from maad.util import mean_dB
import librosa

# Import MOSQITO color sheme [Optional]
from mosqito import COLORS

In [ ]:
data_augmented = pd.read_csv("../data/responses_complete.csv")
# For each audio...
audios_path = "../data/soundscapes_augmented/"
for file in sorted(os.listdir(audios_path)):
        if file.endswith('.mp3') or file.endswith('.wav'):
            print("FILEEEEEEEEEE ", file)
            # audio path
            audio_path=audios_path+file
            # Find the row where the filename matches in the "soundscape" column
            file_split=file.split("_")
            file_fold=int(file_split[1])
            file_participant="ARAUS_"+file_split[3]
            file_stimulus=int(file_split[5].split(".")[0])
            audio_info_aug = data_augmented[data_augmented['fold_r'] == file_fold]
            audio_info_aug = audio_info_aug[audio_info_aug['stimulus_index'] == file_stimulus]
            audio_info_aug = audio_info_aug[audio_info_aug['participant'] == file_participant] # Row of info from responses.csv
            soundscape=audio_info_aug['soundscape'].values[0]
            audio_info_gen = data_generated[data_generated['soundscape_file'] == soundscape] # Row of info form generated.csv
            print("audio sounscape ", soundscape)

            #Get Leq of raw digital audio to calculate audio gain needed
            rawR, fs = load(audio_path, wav_calib=1, ch=1) #R
            rawR_Leq_avg=mean_dB(pressure2leq(rawR, fs, 0.125))
            trueLeq=audio_info_aug["Leq_R_r"].values[0]
            difference=trueLeq-rawR_Leq_avg
            audio_gain=10**(difference/20)
            print(audio_gain, " gain factor")